In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from fake_useragent import UserAgent

In [3]:
ua = UserAgent()
headers = {'User-Agent': ua.random}

base_url = "https://uk.trustpilot.com/review/www.vodafone.co.uk?page="

reviews = []
seen_reviews = set() 


In [5]:
for page in range(1, 2):  # Adjust number of pages as needed
    url = base_url + str(page)
    print(f"Scraping page {page}...")
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    for i, card in enumerate(soup.select('article')):
        if i < 4:
            continue  # 🚫 skip sticky duplicates

        try:
            rating = card.select_one('[data-service-review-rating]').get('data-service-review-rating')
        except:
            rating = None
        
        try:
            review = card.select_one('p.typography_body-l__v5JLj').text.strip()
        except:
            review = None

        try:
             # Try to get full name first
            name_tag = card.select_one('span[data-consumer-name-typography]')
            if name_tag:
                name = name_tag.text.strip()
            else:
                # Fallback to initials if full name missing
                name = card.select_one('span.typography_heading-xs__osRhC').text.strip()
        except:
            name = None

        try:
            location = card.select_one('span.typography_body-m__k2UI7').text.strip()
        except:
            location = None

        try:
            date = card.select_one('time')['datetime']
        except:
            date = None

        reviews.append({
            'Rating': rating,
            'Review': review,
            'Name': name,
            'Location': location,
            'Date': date
        })

    # ✅ Random delay to stay under radar
    time.sleep(random.uniform(2.5, 5.0))

    if page % 100 == 0:
        temp_df = pd.DataFrame(reviews)
        temp_df.to_csv(f"vodafone_backup_pg{page}.csv", index=False)
        print(f"💾 Backup saved at page {page}")
    # Save progress every 100 pages


df = pd.DataFrame(reviews)

Scraping page 1...


In [ ]:
df

,Rating,Review,Name,Location,Date
0,1,"Trying to buy broadband through Uswitch, then ...",Alfie Calas,GB,2025-06-05T22:03:24.000Z
1,5,Sona did a great job either my trade in and co...,Julliette,GB,2025-06-05T21:51:17.000Z
2,1,"One of the worst , if not the worst when it co...",Vlad Ureche,GB,2025-06-05T21:28:36.000Z
3,1,How those people get 4.6 rate that’s a joke my...,Adam Farbotko,GB,2025-06-05T21:14:08.000Z
4,1,Held to ransom by a ‘reputable’ company. Purch...,Chloe,GB,2025-06-05T20:24:59.000Z
...,...,...,...,...,...
9995,1,"absolutely awful, been with vodaphone for 4 mo...",Ross Harker,GB,2025-03-05T11:25:39.000Z
9996,1,Had a great experience- until we decided to mo...,Fern Jones,GB,2025-03-04T22:32:25.000Z
9997,5,Karan and Isaac were both amazing. Give them a...,Evan Calitz,GB,2025-03-04T22:25:37.000Z
9998,5,Thank you mani your amazing,annalisa torciano,IT,2025-03-04T21:50:39.000Z


In [ ]:
df.Location.value_counts()

Location
GB    9837
US      21
FR      14
IE      13
IT       9
ES       8
NL       7
IN       7
AU       6
DE       5
PT       5
CA       5
BE       3
ZA       3
SA       3
CH       3
TR       3
BG       3
RO       2
MA       2
PH       2
UA       2
AE       2
IL       2
HK       2
PL       2
AO       2
AT       1
BD       1
GR       1
ID       1
ZW       1
MY       1
BA       1
PK       1
GI       1
KH       1
SN       1
IM       1
NI       1
EG       1
NG       1
SE       1
MT       1
NO       1
CY       1
MD       1
MX       1
TH       1
NZ       1
IS       1
LV       1
BB       1
LK       1
Name: count, dtype: int64

In [ ]:
df.Review.head(1)

0    Trying to buy broadband through Uswitch, then ...
Name: Review, dtype: object